In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import tensorflow as tf

Neural Network Model for Hyper Parameter Tuning

In [2]:
# Define the true objective function z = f(x, y)
def obj_func(x, y):
    return -(20 * (1 - np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np. pi * y))) + np.exp(1))

In [3]:
# Random Noise that will be added to data Response
ran_err = lambda n, x: np.random.normal(0, x, n)

In [4]:
# Generate synthetic data for training the neural network
def generate_data():
    x = np.random.uniform(-35, 35, 10)
    y = np.random.uniform(-35, 35, 10)
    z = obj_func(x, y)
    return np.vstack((x, y)).T, z

# Generate the data
X, z = generate_data()
z = z + ran_err(z.shape, 0.05)


# Preprocess the data 
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Define the hyperparameter space
hyp_space  = [
    Integer(1, 5, name='num_layers'), # Num of layers in the network (depth)
    Integer(10, 100, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.0001, 0.1, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.5, name='dropout_rate'), # Probability of droping out a neuron
    Integer(10, 100, name='batch_size'), # Num of samples per batch
    Integer(1, 20, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

In [5]:
# Defining the neural network model
# def NN_model(num_layers, num_units, learning_rate, dropout_rate):
#     model = Sequential()
#     model.add(Dense(num_units, activation='relu', input_shape=(2,)))
#     for _ in range(num_layers - 1):
#         model.add(Dense(num_units, activation='relu'))
#         model.add(Dropout(dropout_rate))
#     model.add(Dense(1, activation='linear'))
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
#     return model

def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(2,))
    x = Dense(num_units, activation='relu')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# # Define the objective function to minimize
# @use_named_args(hyp_space)
# def objective(**params):
#     model = KerasRegressor(model=NN_model, **params, verbose=0)
#     return -np.mean(cross_val_score(model, X, z, cv=3, n_jobs=-1, scoring='neg_mean_squared_error'))

# Define the objective function to minimize
@use_named_args(hyp_space)
def objective(**params):
    num_layers = params['num_layers']
    num_units = params['num_units']
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']

    model = NN_model(num_layers, num_units, learning_rate, dropout_rate)

    kfold = KFold(n_splits=3)
    scores = []

    for train_idx, val_idx in kfold.split(X):
        model.fit(X[train_idx], z[train_idx], epochs=epochs, batch_size=batch_size, verbose=0)
        score = model.evaluate(X[val_idx], z[val_idx], verbose=0)
        scores.append(score)
    
    return np.mean(scores)

# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=50, random_state=0, acq_func='EI')

# Output best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters
best_model = NN_model(
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3],
)

Best hyperparameters:
num_layers: 2
num_units: 88
learning_rate: 0.002635264040018728
dropout_rate: 0.4004553759898222
batch_size: 57
epochs: 14


In [6]:
# Train the model
best_model.fit(X, z, epochs=result.x[5], batch_size=result.x[4], verbose=1)

# Function for MC Dropout predictions
def mc_dropout_predictions(model, X, num_samples=100):
    predictions = np.zeros((num_samples, X.shape[0]))
    for i in range(num_samples):
        predictions[i, :] = model(X, training=True).numpy().flatten()
    prediction_mean = predictions.mean(axis=0)
    prediction_std = predictions.std(axis=0)
    return prediction_mean, prediction_std

# Make predictions with MC Dropout
mean, std = mc_dropout_predictions(best_model, X)

# Selecting new points with highest uncertainty
num_new_points = 8  # Num of new points to sample
new_points_indices = np.argsort(std)[-num_new_points:]
new_points = X[new_points_indices]

print("New points to sample:")
print(new_points)

Epoch 1/14


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step - loss: 405.0016
Epoch 2/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 323.3695
Epoch 3/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 249.6107
Epoch 4/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 230.5619
Epoch 5/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 192.7144
Epoch 6/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 137.8011
Epoch 7/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 102.0413
Epoch 8/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 91.2842
Epoch 9/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 83.1355
Epoch 10/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 71.5370
Epoch 11/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 69.6246
Epoch 12/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 61.7392
Epoch 13/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 84.4518
Epoch 14/14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 61.5925
New points to sample:
[[  9.63692505  -3.01086525]
 [  2.40671578  13.46465717]
 [-25.16